In [1]:
import backtrader as bt
import btoandav20 as bto
import json

In [2]:
storekwargs = dict(
    token='b765c1ffaf6594b3e8159c824017f77c-05521b6b0337f73391057d810f8c3727',
    account='101-003-22974849-001',
    practice=True,
    notif_transactions=True,
    stream_timeout=10,
)
store = bto.stores.OandaV20Store(**storekwargs)

datakwargs = dict(
    timeframe=bt.TimeFrame.Minutes,
    compression=1,
    tz='Europe/Berlin',
    backfill=False,
    backfill_start=False,
)
data = store.getdata(dataname="EUR_USD", **datakwargs)

data.resample(
    timeframe=bt.TimeFrame.Minutes,
    compression=1)  # rightedge=True, boundoff=1)

cerebro = bt.Cerebro()
cerebro.adddata(data)
cerebro.setbroker(store.getbroker())


In [3]:
class St(bt.Strategy):

    def __init__(self):
        self.order = None
        self.mybuysignals = {
            "buysignal1": False,
            "buysignal2": False,
            "buysignal3": False,
            "buysignal4": True,
            "buysignal5": False,

        }

    def notify_store(self, msg, *args, **kwargs):
        print(type(msg))
        print(msg)
        #if "clientExtensions" in msg:
        #    o_info = json.loads(msg["clientExtensions"]["comment"])
        #    buytrigger = o_info["buytrigger"]

    def next(self):
        if self.order:
            return

        for k, v in self.mybuysignals.items():
            if v:
                self.order = self.buy(
                    size=1,
                    buytrigger=k
                    )


In [4]:
cerebro.addstrategy(St)
cerebro.run()

<class 'str'>
{'id': '22', 'time': '1679197682.608654429', 'userID': 22974849, 'accountID': '101-003-22974849-001', 'batchID': '22', 'requestID': '61086261085660053', 'type': 'MARKET_ORDER', 'instrument': 'EUR_USD', 'units': '1.0', 'timeInForce': 'FOK', 'positionFill': 'DEFAULT', 'reason': 'CLIENT_ORDER', 'clientExtensions': {'id': '1679197678.973387-1', 'comment': '{"buytrigger": "buysignal4"}'}}
<class 'str'>
{'id': '23', 'time': '1679197682.608654429', 'userID': 22974849, 'accountID': '101-003-22974849-001', 'batchID': '22', 'requestID': '61086261085660053', 'type': 'ORDER_CANCEL', 'orderID': '22', 'clientOrderID': '1679197678.973387-1', 'reason': 'MARKET_HALTED'}


KeyboardInterrupt: 